In [80]:

import pandas as pd
import numpy as np
from datetime import datetime



first_data = pd.read_csv("Final_CE_10042023_V3.csv").set_index(keys=["record","uuid"]).sort_index()
first_data.head()


,,date,markers,vboost,hboost,hQC_FINAL,S1,S2,hAGE,hGender,S3r1,...,Q18r3,Q18r4,Q18r5,Q19r1,Q19r2,Q20,Q22,Q23,Q24,Q26
record,uuid,,,,,,,,,,,,,,,,,,,,,
17,pdxxkfp5z4x0sjh9,06-Sep-23,"qualified,ver_9,/MD_Maxdiff/ver_9,/overall/qua...",1,0,0,32,1,1.0,1,0,...,2,5,3,5,5,1,1,2,1,1
19,8sxrhwk566x44rjn,06-Sep-23,"qualified,/boost2/genAGE_2,ver_114,/MD_Maxdiff...",3,2,1,40,1,4.0,1,0,...,2,2,3,2,2,1,1,2,2,1
20,cy214wpwe85ednz6,06-Sep-23,"qualified,/gen_pop_dtv/DTV_SAT_USERS,/overall/...",1,0,0,66,1,NaN,1,0,...,2,5,3,5,2,1,1,2,2,3
21,57btpx6f2h0hk643,06-Sep-23,"qualified,/gen_pop_samp/genAGE_3,/overall/qual...",1,0,0,69,1,3.0,1,0,...,1,1,3,5,1,2,3,2,2,3
23,cd1yajua6cuseqk6,06-Sep-23,"qualified,/MD_Maxdiff/ver_200,ver_200,/overall...",1,0,1,66,2,3.0,2,0,...,1,2,5,6,1,3,1,2,2,3


In [81]:
def clean_blank_and_convert_to_numeric(first_data):
    exclude_cols = ['date','markers','record','uuid']
    cols_to_convert = first_data.columns.difference(exclude_cols)
    first_data[cols_to_convert] = first_data[cols_to_convert].replace({' ':np.nan,'':np.nan})
    first_data[cols_to_convert] = first_data[cols_to_convert].apply(pd.to_numeric,errors='coerce')
    return first_data

In [82]:
first_data = clean_blank_and_convert_to_numeric(first_data)
first_data.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3439 entries, (np.int64(17), 'pdxxkfp5z4x0sjh9') to (np.int64(20976), 'wdd0auqsjehcwu1u')
Columns: 250 entries, date to Q26
dtypes: float64(47), int64(201), object(2)
memory usage: 6.9+ MB


In [83]:

print(first_data['hAGE'].unique())
print(first_data['hAGE'].value_counts())
#print(first_data['Q16'].apply(type).value_counts())



[ 1.  4. nan  3.  2.]
hAGE
3.0    1362
2.0    1120
1.0     955
4.0       1
Name: count, dtype: int64


In [84]:
class TabGenerator:
    def __init__(self,first_data,question_var,question_text,base_text,code_label_map,table_number,study_name,client_name,month,year,question_type,mean_var,filter_condition=None):

        self.df=first_data.copy()
        self.question_var = question_var
        self.question_text = question_text
        self.base_text = base_text
        self.codes = code_label_map
        self.table_number = table_number
        self.study_name = study_name
        self.client_name = client_name
        self.month = month
        self.year = year
        self.question_type = question_type
        self.mean = mean_var
        self.filter_condition = filter_condition

    def generate(self):
        if self.filter_condition:
            df_filtered = self.df.query(self.filter_condition)
        else:
            df_filtered = self.df

        base_n = df_filtered.shape[0]
        result_row = []

        result_row.append([self.client_name,""])
        result_row.append([self.study_name,""])
        result_row.append([f"{self.month} {self.year}",""])
        result_row.append([f"Table {self.table_number}",""])
        result_row.append([self.question_text,""])
        result_row.append(["",""])
        result_row.append([f"Base: {self.base_text}",base_n])

        total_count = 0
        if self.question_type=="multi":
            for var, label in self.codes.items():
                count = (df_filtered[var] == 1).sum()
                percent = (count/base_n *100) if base_n > 0 else 0
                result_row.append([label,count])
                result_row.append(["",f"{percent:.2f}%"])
                total_count +=count
                total_percent = (total_count/base_n) * 100 if base_n > 0 else 0
            if total_count < base_n:
                result_row.append(["No Answer",base_n - total_count])
                result_row.append(["",f"{(base_n - total_count)/base_n * 100:.2f}%"])
            result_row.append(["Sigma",total_count])
            result_row.append(["",f"{total_percent:.2f}%"])
            result_row.append(["",""])
                
        elif self.question_type=="single":
            for code,label in self.codes.items():
                count = (df_filtered[self.question_var]==code).sum()
                percent = (count/base_n) * 100 if base_n > 0 else 0
                result_row.append([label,count])
                result_row.append(["",f"{percent:.2f}%"])        
            total_count = sum((df_filtered[self.question_var]==code).sum() for code in self.codes)
            total_percent = (total_count/base_n) * 100 if base_n > 0 else 0
            no_answer = base_n - total_count
            if total_count < base_n:
                result_row.append(["No Answer",no_answer])
                result_row.append(["",f"{(no_answer)/base_n * 100:.2f}%"])
            total_sigma = total_count + no_answer
            total_sigma_percent = (total_sigma/base_n) * 100 if base_n > 0 else 0
            result_row.append(["Sigma",total_sigma])
            result_row.append(["",f"{total_sigma_percent:.2f}%"])
            if self.mean and self.mean in df_filtered.columns:
                stats_var = df_filtered[self.mean].dropna()
                if not stats_var.empty:
                    mean_val = df_filtered[self.mean].mean()
                    median_val = df_filtered[self.mean].median()
                    std_val = df_filtered[self.mean].std()
                    sem_val = df_filtered[self.mean].sem()                
                result_row.append([f"Mean",f"{mean_val:.2f}"])
                result_row.append([f"Std. Dev.",f"{std_val:.2f}"])
                result_row.append([f"Std. Err.",f"{sem_val:.2f}"])
                result_row.append([f"Median",f"{median_val:.2f}"])
            result_row.append(["",""])
            
        else:
            return "Question not recoznized"
        
        return pd.DataFrame(result_row,columns=["",""])

            






In [85]:
tabs_config = [
    {
        "question_var": "hGender",
        "question_text": "hGender. What is your gender?",
        "base_text": "Total Respondents",
        "codes": {1:"Male",2:"Female"},
        "base_filter":None,
        "question_type" : "single",
        "mean_var": None
        
    },
    {
        "question_var": ["S3r1"	,"S3r2"	,"S3r3"	,"S3r4"	,"S3r5"	,"S3r6"	,"S3r7"	,"S3r8"	,"S3r9"],
        "question_text": "S3. We are interested in getting feedback from people who work in a variety of areas. Do you, or does anyone in your family or household work in any of the following industries?",
        "codes":{

            "S3r1":"In advertising, public relations, or journalism",
            "S3r2":"In marketing or market research",
            "S3r3":"For a telecommunications company",
            "S3r4":"In financial services/banking",
            "S3r5":"In education",
            "S3r6":"In hospitality/travel/tourism",
            "S3r7":"In government/public service",
            "S3r8":"In retail",
            "S3r9":"None of the above "
        },
        "base_text": "Total Respondents",
        "base_filter":None,
        "question_type" : "multi",
        "mean_var": None
        
    },

    {
        "question_var": "hAGE",
        "question_text": "hAge. What is your age group?",
        "codes":{1:"18-29",2:"30-39",3:"40-49",4:"50+"},
        "base_text": "Total Respondents",
        "base_filter":None,
        "question_type" : "single",
        "mean_var": "S1"
        
    }
]

study_name = "DTV-010 Feature Prioritization"
client_name = "PEERLESS INSIGHTS"
now = datetime.now()
month =  now.strftime("%B")
year = now.year





results = []
 
for i, table in enumerate(tabs_config, start=1):
    tg = TabGenerator(
        client_name=client_name,
        study_name=study_name,
        month=month,
        year=year,
        first_data=first_data,
        question_var=table["question_var"],
        question_text=table["question_text"],
        base_text=table["base_text"],
        code_label_map=table["codes"],
        question_type=table["question_type"],
        table_number=i,
        mean_var=table["mean_var"],
        filter_condition=table["base_filter"]
    )
    
    result_df = tg.generate()
    results.append(result_df)

final_df = pd.concat(results,ignore_index=True)
final_df.to_csv("tabs_output.csv",index=False)